# [Scrapy Beginners Series Part 2: Cleaning Dirty Data & Dealing With Edge Cases](https://scrapeops.io/python-scrapy-playbook/scrapy-beginners-guide-cleaning-data/)

# Scrapy 初心者向けシリーズ パート 2: ダーティ データのクリーニングとエッジ ケースへの対処